In [1]:
import random, numpy as np, pandas as pd
from matplotlib import pyplot as plt
alphap = 2.5
probp = 0.75
expon = 2 * alphap - 1
slope = 0.784498

In [2]:
def betadist(alpha):
    """gives a random number from beta distribution"""
    return random.betavariate(alpha,alpha)

In [3]:
def decision(probability):
    """
    decides with a given probability whether to keep the right part
    """
    if float(probability) > random.random():
        return True
    else: 
        return False

In [4]:
def splitting(segment):
    """
    splits a given segment. left and right are endpoints of the segment
    returns : 
        xL -> length of the left segment
        xR -> length of the right segment
        flag -> keeping the right segment
        xLp, xRp -> probability(unnormalized) for being selected
        change -> change of normalization const
    """
    xL = segment * betadist(alphap)
    xR = segment - xL
    flag = decision(probp)
    xLp = xL**expon
    xRp = xR**expon
    change = xLp + xRp - segment**expon
    return xL, xR, flag, xLp, xRp, change

In [5]:
# def pickindex(frag_prob, frag_prob_sum):
#     """
#     picks up a segment to be subsequently split
#     """
#     r = random.uniform(0, frag_prob_sum)
#     sum_ = 0
#     for index in range(len(frag_prob)):
#         sum_ += frag_prob[index]
#         if sum_ < r:
#             continue
#         else:
#             return index

In [6]:
def pickindex(frag_prob, frac_sum):
    """
    picks up a segment to be subsequently split
    """
    r = random.uniform(0,1)
    
    if r <= frac_sum:
        sum_ = 0
        for index in range(len(frag_prob)):
            sum_ += frag_prob[index]
            if sum_ < r:
                continue
            else:
                return index

In [7]:
def number_length(segment_lengths,flags):
    
    N = 0
    M = 0

    for i in range(len(flags)):
        if flags[i]:
            N += 1
            M += segment_lengths[i]
            pass
        pass
    return N, M

In [8]:
fractal_dim = slope

def fractal_length(segment_lengths,flags):

    M_frac = 0

    for i in range(len(flags)):
        if flags[i]:
            M_frac += segment_lengths[i]**fractal_dim
            pass
        pass
    return M_frac

In [9]:
def realization_value(total_iteration, min_iteration, iteration_step, data_point = False):
    
    lengths = [1.]
    flags = [True]
    frag_prob = [1.] # raw probability, not normalized
    #frag_prob_sum = 1.0 # normalization const
    frac_sum = 1.0

    iteration_list = list(range(min_iteration, total_iteration + 1, iteration_step))
    
    M_realization = []
    
    for i in range(total_iteration + 1):
        
        #index = pickindex(frag_prob, frag_prob_sum)
        index = pickindex(frag_prob, frac_sum)
        
        if index != None and flags[index] == True:

            xL, xR, flag, xLp, xRp, change = splitting(lengths[index])
            
            lengths[index] = xL
            lengths.append(xR)
            flags.append(flag)
            frag_prob[index] = xLp 
            frag_prob.append(xRp)
            #frag_prob_sum += change
            frac_sum += change
            pass
        
        if i+1 in iteration_list:
            M_frac = fractal_length(lengths,flags)
            M_realization.append(M_frac)
        pass 
      
    if data_point:
        print(np.array(M_realization))
    
    else:
        M_frac = fractal_length(lengths,flags)
        return M_frac

In [10]:
def ensemble_average(total_iteration, ensemble_size):

    M_ensemble = 0
    
    for i in range(ensemble_size):
        M_list = realization_value(total_iteration, total_iteration, total_iteration)
        M_ensemble += M_list
        pass
    
    M_average = M_ensemble/ensemble_size
    
    print(M_average)

In [11]:
for i in range(5):
    realization_value(1_000_000, 100_000, 100_000, data_point = True)

[0.61201153 0.61201153 0.62548265 0.63727278 0.63727278 0.63727278
 0.63727278 0.63727278 0.65007599 0.65927049]
[1.013736   1.03220176 0.98746243 1.01193821 1.00108689 0.98091026
 0.98091026 0.98091026 1.00929577 1.00929577]
[0.53496045 0.54959849 0.5633264  0.5633264  0.53613722 0.53613722
 0.53613722 0.54518704 0.54518704 0.54518704]
[0.8922333  0.76735088 0.7995134  0.83168375 0.84403359 0.84403359
 0.86927202 0.83215098 0.83215098 0.84231601]
[1.01363284 1.00994634 1.02029101 0.98539944 0.9936524  1.00057202
 1.02057081 1.02879132 1.02879132 1.02879132]


In [12]:
for i in range(10):
    a_ = 1_000*(i+1)
    ensemble_average(a_, 100_000)

1.0005619898269968
1.00145062342772
1.0014333475263386
1.001975700165262
0.9987970630189374
1.0004029943889197
1.0003688552175234
1.0009364507407603
1.0014618198095058
0.9993938586360912
